In [1]:
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time

In [2]:
import pandas as pd

In [3]:
#chromedriver_path = "C:\Program Files (x86)\chromedriver.exe"

In [4]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [5]:
# technology jobs
url1 = 'https://www.kpmguscareers.com/job-search/?career-level-parents=Experienced%7C&career-level=&keyword=technology&location=&radius=50&spage='
url2 = ''

In [7]:
def kpmg_get_jobs(url1, url2):
    """
    retrieve job title and job link from every page in the URL
    """
    job_title=[]
    job_link=[]
    page_num = 0

    driver=webdriver.Chrome(options = chrome_options)
    URL = url1 + str(page_num) + url2
    driver.get(URL)
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    while soup.find_all('div', {'class': 'h5 text-dark-grey'}) :
        job_title.extend([td.text for td in soup.find_all('div', {'class': 'h5 text-dark-grey'})])
        job_link.extend(['https://www.kpmguscareers.com' + td['href'] for td in soup.findAll("a", {"class": "box-shadow d-block"})])
        driver.quit()

        page_num += 1
        driver=webdriver.Chrome(options = chrome_options)
        URL = url1 + str(page_num) + url2
        driver.get(URL)
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    driver.quit() 
    return job_title,job_link

In [8]:
job_title, job_link = kpmg_get_jobs(url1, url2)

In [33]:
def remove_space(s):
  s_new = s.replace('\n', '').replace('\xa0', '')
  while '  ' in s_new:
    s_new = s_new.replace('  ', ' ')
  return s_new

In [17]:
#job_title = [remove_space(job) for job in job_title]

In [40]:
# create a dataframe that contains job titles and links for all job categories
df_title_link = pd.DataFrame(zip(job_title, job_link), columns=['JOB_TITLE', 'JOB_LINK'])

In [42]:
# drop the duplicates
df_title_link = df_title_link.drop_duplicates()

929


## Extract job descriptions and qualifications for each job

In [22]:
# retrieve job qualifications and descriptions
def job_description(title, link):
    qualifications = []
    description = []
    jobtitle = []
    joblink = []

    driver=webdriver.Chrome('chromedriver',options=chrome_options)
    for i in range(len(link)):
        URL=link[i]
        driver.get(URL)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        s = ''
        d = ''

        try:
          tag = soup.find("h3", text='Qualifications:').findNextSibling("ul").find_all('li')
          if tag:
            for t in tag:
                s = s + '. ' + t.text
        except: pass

        try:
          tag = soup.find("h3", text='Responsibilities:').findNextSibling("ul").find_all('li')
          if tag:
            for t in tag:
                d = d + '. ' + t.text
        except: pass
        
        qualifications.append(s)
        description.append(d)   
        jobtitle.append(title[i])
        joblink.append(link[i])
        
    driver.quit()            
 
    return jobtitle, link, qualifications, description

In [30]:
# retrieve the qualification for each job.
title, link, qual, descrp = job_description(df_title_link['JOB_TITLE'].values, df_title_link['JOB_LINK'].values)

49 no description
91 no qual
91 no description
186 no qual
186 no description
207 no qual
228 no qual
228 no description
235 no description
403 no qual
403 no description
418 no qual
418 no description
483 no qual
483 no description
507 no description
530 no qual
530 no description
547 no qual
547 no description
852 no qual
852 no description
888 no qual
895 no qual
904 no qual
904 no description


In [47]:
qual_cleaned = [remove_space(q) for q in qual]
descrp_cleaned = [remove_space(d) for d in descrp]

In [49]:
#create a dataframe from basic qualification and preferred qualification
df = pd.DataFrame(zip(title, qual_cleaned, link, descrp_cleaned))
df.columns = ['TITLE', 'QUALIFICATIONS', 'LINK', 'DESCRIPTION']
df['COMPANY'] = 'KPMG'
df = df.iloc[:, [4, 0, 1, 2, 3]]

,COMPANY,TITLE,QUALIFICATIONS,LINK,DESCRIPTION
0,KPMG,Cyber Security Manager - Remote,. Minimum five years of recent experience mana...,https://www.kpmguscareers.com/jobdetail/?jobId...,. Oversee the application vulnerability respon...
1,KPMG,Temporary Help Desk Representative,. Minimum one year of recent experience in Ser...,https://www.kpmguscareers.com/jobdetail/?jobId...,. Provide level one phone/chat support; meet/e...
2,KPMG,"Manager, Microsoft 360 Program Operations Netw...",. Minimum three years of recent experience in ...,https://www.kpmguscareers.com/jobdetail/?jobId...,. Manage Digital Investment Hub: Project Manag...
3,KPMG,"Associate or Senior Associate, Global Trade Te...",. Minimum two years of recent experience for a...,https://www.kpmguscareers.com/jobdetail/?jobId...,. Support implementation of Global Trade Techn...
4,KPMG,Audit Manager,. Minimum of five years of current and/or rece...,https://www.kpmguscareers.com/jobdetail/?jobId...,. Manage and supervise the execution of extern...


In [51]:
# look for empty QUALIFICATION entries
def get_empty(df, colname) :
  empty_idx=[]
  for i in range(len(df[colname])):
    count = len(df[colname][i])
    if count<=10 : empty_idx.append(i)
    else: pass
  return empty_idx

empty_mini_q = get_empty(df, 'QUALIFICATIONS')
print(len(empty_mini_q))

# drop the empty Qualification entries
df.drop(empty_mini_q, inplace=True)

13


(916, 5)

In [54]:
#remove the duplicated jobs
df_nodup = df.drop_duplicates()
print("There are {} jobs from KPMG.".format(df_nodup.shape[0]))

df_nodup.to_csv('kpmg_technology_jobs_cleaned.csv')